In [2]:
pip install 'apache-beam[gcp]'

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install google-cloud-pubsub

Note: you may need to restart the kernel to use updated packages.


In [15]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.io.gcp.pubsub import ReadFromPubSub

def run():
    # Define pipeline options for Dataflow execution
    options = PipelineOptions(
        project='dataplatform-438202',  # Your GCP project ID
        region='us-central1',           # Your GCP region
        temp_location='gs://your-temp-bucket/temp',  # Temp GCS location
        streaming=True,                 # Enable streaming mode
        runner='DataflowRunner',        # Set Dataflow as the runner
        service_account_email='27230151676-compute@developer.gserviceaccount.com'  # Service account email
    )

    # Create the Beam pipeline
    with beam.Pipeline(options=options) as p:
        # Step 1: Read data from Pub/Sub topic
        raw_data = (
            p
            | 'Read from Pub/Sub' >> ReadFromPubSub(topic='projects/dataplatform-438202/topics/iot-data')
        )

        # Step 2: Write raw data to Google Cloud Storage
        (raw_data
         | 'Write Raw to GCS' >> beam.io.WriteToText(
                'gs://raw-iot-data-bucket/raw-data/output', 
                shard_name_template='-SS-of-NN',  # Create multiple files (shards)
                file_name_suffix='.json',
                num_shards=5  # Adjust the number of output files
            )
        )

if __name__ == '__main__':
    run()


ValueError: GroupByKey cannot be applied to an unbounded PCollection with global windowing and a default trigger